In [1]:
# libraries
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
from tensorflow import math as TFmath
import math,os,shutil
from prettytable import PrettyTable
import scipy.stats as stats
from sklearn.model_selection import train_test_split

print("TensorFlow version:", tf.__version__)

# 检查 GPU 是否可用
gpus = tf.config.list_physical_devices('GPU')
print("Num GPUs Available: ", len(gpus))
print("Physical GPUs:", gpus)

if len(gpus) == 0:
    print("\n[警告] 未检测到 GPU。可能有以下原因：")
    print("1. 未安装 NVIDIA 显卡驱动或驱动版本过低。")
    print("2. 未安装 CUDA Toolkit (需 11.2) 或 cuDNN (需 8.1) - 针对 TF 2.10 Windows版。")
    print("3. 如果安装了 TF > 2.10，Windows Native GPU 支持已被移除，需使用 WSL2。")
else:
    # 显存按需分配，防止在这步占满显存导致后续报错
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("GPU 显存设置为按需分配 (Memory Growth Enable)")
    except RuntimeError as e:
        print(e)

tf.random.set_seed(256)

TensorFlow version: 2.8.0
Num GPUs Available:  1
Physical GPUs: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
GPU 显存设置为按需分配 (Memory Growth Enable)


In [2]:
# Full paths to training data
TRAIN_FEATURE_PATH = "../Features/Train/train_features.csv"
TRAIN_LABEL_PATH   = "../Features/Train/train_labels.csv"

# Full path to test data
TEST_FEATURE_PATH = "../Features/Test/test_features.csv"

# Output folders
figure_prepath = "../figures/"
model_prepath = "../model/sev_se_block_fe/"

# Create output folders if not existed
os.makedirs(figure_prepath, exist_ok=True)
os.makedirs(model_prepath, exist_ok=True)

# Plotting configuration
figureFrontSize = 12
figureName_post = "_test.png"


In [3]:
### helper labels - fixed
Numchannels = 95
num_inputFeatures = Numchannels * 2 + 4 # target_gain, target_gain_tilt, EDFA_input_power_total, EDFA_output_power_total
labels = {"gainValue":'target_gain',
          "EDFA_input":'EDFA_input_power_total',
          "EDFA_output":'EDFA_output_power_total',
          "inSpectra":'EDFA_input_spectra_',
          "WSS":'DUT_WSS_activated_channel_index_',
          "result":'calculated_gain_spectra_'}
inSpectra_labels = [labels['inSpectra']+str(i).zfill(2) for i in range(0,Numchannels)]
onehot_labels = [labels['WSS']+str(i).zfill(2) for i in range(0,Numchannels)]
result_labels = [labels['result']+str(i).zfill(2) for i in range(0,Numchannels)]
preProcess_labels = [labels['EDFA_input'],labels['EDFA_output']]
preProcess_labels.extend(inSpectra_labels)

In [4]:
def dB_to_linear(data):
  return np.power(10,data/10)

def linear_TO_Db(data):
  result = 10*np.log10(data).to_numpy()
  return result[result != -np.inf]

def linear_TO_Db_full(data):
  result = 10*np.log10(data).to_numpy()
  result[result == -np.inf] = 0
  return result

def divideZero(numerator,denominator):
  with np.errstate(divide='ignore'):
    result = numerator / denominator
    result[denominator == 0] = 0
  return result

In [5]:
# # === 新增：针对 SHB 和全局统计的特征工程函数 ===
# def add_shb_features(df):
#     df_eng = df.copy()
    
#     # 1. 提取光谱列 (Input Spectra columns) 和 WSS 状态列
#     # 模糊匹配列名
#     spectra_cols = [c for c in df.columns if 'EDFA_input_spectra' in c]
#     wss_cols = [c for c in df.columns if 'DUT_WSS_activated_channel_index' in c]
    
#     # 确保按顺序排列
#     spectra_cols.sort()
#     wss_cols.sort()
    
#     # 2. SHB 关键统计特征 (Global Statistics)
#     # 标准差(std): 直接反映光谱的平坦程度。std 越大，烧孔风险越高。
#     df_eng['shb_std']  = df_eng[spectra_cols].std(axis=1)
#     # 均值(mean)与总和(sum): 反映总输入功率水平
#     df_eng['shb_mean'] = df_eng[spectra_cols].mean(axis=1)
# #     df_eng['shb_sum']  = df_eng[spectra_cols].sum(axis=1) 
# #     # 最大值(max): 局部强光点是 SHB 的主要诱因
# #     df_eng['shb_max']  = df_eng[spectra_cols].max(axis=1) 
    
# #     # 3. 活跃信道数 (Load)
# #     # 信道越少，单信道分到的功率可能越高，容易产生非线性效应
# #     df_eng['feat_active_channel_count'] = df_eng[wss_cols].sum(axis=1)
    
# #     # 4. 光谱重心 (Spectral Center of Mass)
# #     # 衡量输入光功率是集中在长波(红移)还是短波(蓝移)，影响增益倾斜(Tilt)
# #     indices = np.arange(len(spectra_cols))
# #     # 矩阵乘法快速计算加权和
# #     weighted_sum = df_eng[spectra_cols].dot(indices)
# #     # 防止除零
# #     total_p = df_eng[spectra_cols].sum(axis=1) + 1e-9
# #     df_eng['shb_center'] = weighted_sum / total_p
    
# #     return df_eng
# def add_shb_features(df):
#     df_eng = df.copy()
    
#     # 1. 提取光谱列 (Input Spectra columns) 和 WSS 状态列
#     spectra_cols = [c for c in df.columns if 'EDFA_input_spectra' in c]
#     wss_cols = [c for c in df.columns if 'DUT_WSS_activated_channel_index' in c]
#     spectra_cols.sort()
#     wss_cols.sort()
    
#     # === SHB 关键统计特征 ===
#     df_eng['shb_std']  = df_eng[spectra_cols].std(axis=1)
#     df_eng['shb_mean'] = df_eng[spectra_cols].mean(axis=1)
#     df_eng['shb_sum']  = df_eng[spectra_cols].sum(axis=1)
#     df_eng['shb_max']  = df_eng[spectra_cols].max(axis=1)
    
#     # === 活跃信道数 ===
#     df_eng['feat_active_channel_count'] = df_eng[wss_cols].sum(axis=1)
    
#     # === 光谱重心 ===
#     indices = np.arange(len(spectra_cols))
#     weighted_sum = df_eng[spectra_cols].dot(indices)
#     total_p = df_eng[spectra_cols].sum(axis=1) + 1e-9
#     df_eng['shb_center'] = weighted_sum / total_p
    
#     # === 物理特征 ===
#     # 线性域总功率 (mW)
#     mw_df = np.power(10, df_eng[spectra_cols] / 10.0)
#     df_eng['total_power_mW'] = mw_df.sum(axis=1)
#     # 动态范围 (Dynamic Range)
#     df_eng['power_span'] = df_eng[spectra_cols].max(axis=1) - df_eng[spectra_cols].min(axis=1)
    
#     return df_eng

import numpy as np
import pandas as pd
from scipy.stats import skew, kurtosis
def add_shb_features(df):
    """
    全能版特征工程函数：集成噪声截断、时域统计、频域FFT、梯度特征、线性域特征
    """
    # 1. 拷贝原始数据
    df_eng = df.copy()
    
    # 2. 自动提取列名
    spectra_cols = [c for c in df.columns if 'EDFA_input_spectra' in c]
    wss_cols = [c for c in df.columns if 'DUT_WSS_activated_channel_index' in c]
    spectra_cols.sort()
    wss_cols.sort()
    
    # === [关键策略 1] 噪声底截断 (Noise Floor Clipping) ===
    # 必须最先执行！清洗掉 -60dBm 以下的随机噪声，防止它们污染后续的均值/方差计算
    # 阈值建议 -55.0，底噪设为 -60.0
    print("  -> Performing Noise Clipping (< -55dBm to -60dBm)...")
    spectra_matrix = df_eng[spectra_cols].values
    spectra_matrix = np.where(spectra_matrix < -55.0, -60.0, spectra_matrix)
    # 将清洗后的数据写回 DataFrame (让模型直接看到干净的光谱)
    df_eng[spectra_cols] = spectra_matrix 
    
    # === 3. 使用字典收集所有新特征 (解决 PerformanceWarning 碎片化问题) ===
    new_feats = {}
    
    # --- A. 基础统计特征 (Based on Clipped Data) ---
    # 使用 numpy 向量化计算，比 pandas apply 快得多
    new_feats['shb_std']  = np.std(spectra_matrix, axis=1)
    new_feats['shb_mean'] = np.mean(spectra_matrix, axis=1)
    new_feats['shb_sum']  = np.sum(spectra_matrix, axis=1)
    new_feats['shb_max']  = np.max(spectra_matrix, axis=1)
    
    # --- B. 差分特征 (Gradient/Shape Features) ---
    # 捕捉光谱的局部斜率和曲率，对 Bi-LSTM/CNN 极有帮助
    # 一阶差分 (Slope)
    grad1 = np.diff(spectra_matrix, axis=1)
    new_feats['grad1_mean_abs'] = np.mean(np.abs(grad1), axis=1) # 平均抖动
    new_feats['grad1_max'] = np.max(np.abs(grad1), axis=1)       # 最大突变
    new_feats['grad1_std'] = np.std(grad1, axis=1)               # 形状复杂度
    
    # 二阶差分 (Curvature)
    grad2 = np.diff(grad1, axis=1)
    new_feats['grad2_mean_abs'] = np.mean(np.abs(grad2), axis=1) # 平均弯曲度
    
    # --- C. WSS 活跃信道 ---
    # 这里直接用原始 df 的列即可
    new_feats['feat_active_channel_count'] = df_eng[wss_cols].sum(axis=1)
    
    # --- D. 光谱重心 (Spectral Centroid) ---
    indices = np.arange(len(spectra_cols))
    # 使用点积加速加权求和
    weighted_sum = np.dot(spectra_matrix, indices) 
    total_p = np.sum(spectra_matrix, axis=1) + 1e-9
    new_feats['shb_center'] = weighted_sum / total_p
    
    # --- E. 物理特征 (线性域 Linear Domain) ---
    # 将 dBm 转为 mW，更能反映真实的物理能量叠加
    mw_matrix = np.power(10, spectra_matrix / 10.0)
    
    new_feats['total_power_mW'] = np.sum(mw_matrix, axis=1)
    # 偏度 (Skew): 能量偏左还是偏右 (SRS效应指示器)
    new_feats['power_skew'] = skew(mw_matrix, axis=1)
    # 峰度 (Kurtosis): 光谱是尖锐的还是平坦的
    new_feats['power_kurt'] = kurtosis(mw_matrix, axis=1)
    # 动态范围
    new_feats['power_span'] = np.max(spectra_matrix, axis=1) - np.min(spectra_matrix, axis=1)
    
    # --- F. FFT 频域特征 (Frequency Domain) ---
    # 捕捉周期性纹波 (Ripple)
    fft_vals = np.fft.fft(spectra_matrix, axis=1)
    fft_abs = np.abs(fft_vals)
    
    new_feats['fft_mean'] = np.mean(fft_abs, axis=1)
    new_feats['fft_std'] = np.std(fft_abs, axis=1)
    new_feats['fft_max'] = np.max(fft_abs, axis=1)
    
    # 提取 FFT 低频分量 (前5个，代表整体轮廓)
    for i in range(1, 6): # 跳过直流分量 0
        new_feats[f'fft_component_{i}'] = fft_abs[:, i]
        
    # === 4. 一次性合并 ===
    # 将字典转换为 DataFrame，并与原数据拼接
    # 这是避免 "DataFrame is highly fragmented" 警告的最佳实践
    new_feats_df = pd.DataFrame(new_feats, index=df_eng.index)
    
    df_final = pd.concat([df_eng, new_feats_df], axis=1)
    
    return df_final

In [6]:
### train and loss function
def custom_loss(y_actual,y_pred):
  # calculate the loaded channel numbers for each batch
  # batch default is [batch size=32, outputchannel number]
  loaded_size = tf.dtypes.cast(TFmath.count_nonzero(y_actual), tf.float32)
  # turn unloaded y_pred prediction to zero
  y_pred_cast_unloaded_to_zero = TFmath.divide_no_nan(TFmath.multiply(y_pred,y_actual),y_actual)
  # error [unloaded,unloaded,loaded,loaded]: y_pred = [13->0,15->0,18.5,18.3], y_actual = [0,0,18.2,18.2]
  error = TFmath.abs(TFmath.subtract(y_pred_cast_unloaded_to_zero,y_actual))
  # custom_loss = (0.3+0.2) / 2
  custom_loss = TFmath.divide(TFmath.reduce_sum(error),loaded_size)
  return custom_loss

def custom_loss_L2(y_actual,y_pred):
  loaded_size = tf.dtypes.cast(TFmath.count_nonzero(y_actual), tf.float32)
  y_pred_cast_unloaded_to_zero = TFmath.divide_no_nan(TFmath.multiply(y_pred,y_actual),y_actual)
  error = TFmath.square(TFmath.subtract(y_pred_cast_unloaded_to_zero,y_actual))
  custom_loss = TFmath.sqrt(TFmath.divide(TFmath.reduce_sum(error),loaded_size))
  return custom_loss
    

In [7]:
def se_block(input_tensor, ratio=16):
    """
    Squeeze and Excitation Block for Dense Layers (Feature Reweighting).
    Explicitly models interdependencies between channels (features).
    """
    # Input shape: (Batch, input_dim)
    input_dim = input_tensor.shape[-1]
    
    # Squeeze: Global Information Aggregation 
    # For Dense layers, the input itself is the global descriptor. 
    # So we proceed directly to Excitation.
    
    # Excitation: Adaptive Recalibration
    # 1. Bottleneck
    x = layers.Dense(input_dim // ratio, use_bias=False, activation='relu')(input_tensor)
    # 2. Rescale
    x = layers.Dense(input_dim, use_bias=False, activation='sigmoid')(x)
    
    # Scale: Reweight input features
    return layers.Multiply()([input_tensor, x])

def designed_DNN_model(outputNum, X_data=None):
  # === 残差学习 (Residual Learning) ===
  # 原理：模型只预测“实际增益”与“目标增益(target_gain)”之间的差值(Residual)。
  # 最终输出 = target_gain + DNN预测的残差。
  # 这样可以显著降低学习难度，因为大部分增益主要由 target_gain 决定，DNN 只需要微调。

  inputs = layers.Input(shape=(num_inputFeatures,))
  
  # 1. 提取 target_gain
  # 假设输入特征的第0列是 target_gain (根据 pd.read_csv(...).iloc[:, 3:] 的顺序)
  target_gain = layers.Lambda(lambda x: tf.expand_dims(x[:, 0], -1), name='extract_target_gain')(inputs)
  
  x = inputs

  # 2. 标准化 (可选，建议加上)
  if X_data is not None:
      normalizer = layers.Normalization(axis=-1)
      normalizer.adapt(X_data)
      x = normalizer(inputs)

  # === SE-Block 1: Initial Feature Reweighting ===
  # 让模型在进入深层之前，先根据全局信息自动调整输入特征的权重
  x = se_block(x, ratio=8) 

  # 3. DNN 主体 (预测残差)
  # 结构建议：Dense -> BN -> ReLU -> Dropout
  
  # 第一层 (保留原始)
  x = layers.Dense(num_inputFeatures, activation='silu')(x)
  
  # 第二层
  x = layers.Dense(256)(x)
#   x = layers.BatchNormalization()(x)
  x = layers.Activation('silu')(x)
  x = layers.Dropout(0.1)(x) # 增加 Dropout, 比率设为 0.2
  
  # === SE-Block 2: Intermediate Feature Refinement ===
  x = se_block(x, ratio=16)

  # 第三层
  x = layers.Dense(128)(x)
#   x = layers.BatchNormalization()(x)
  x = layers.Activation('silu')(x)
  x = layers.Dropout(0.1)(x) # 增加 Dropout

  # 第四层
  x = layers.Dense(128)(x)
#   x = layers.BatchNormalization()(x)
  x = layers.Activation('silu')(x)
  x = layers.Dropout(0.1)(x) # 增加 Dropout

  # 第五层
  x = layers.Dense(128)(x)
#   x = layers.BatchNormalization()(x)
  x = layers.Activation('silu')(x)
  x = layers.Dropout(0.05)(x) # 最后一层 Dropout 稍微小点

  
  residual = layers.Dense(outputNum, name='predicted_residual')(x)
  
  # 4. 残差连接
  outputs = layers.Add(name='add_residual_to_target')([residual, target_gain])
  
  model = keras.Model(inputs=inputs, outputs=outputs)

  model.compile(loss=custom_loss_L2, # custom_loss
                optimizer=tf.keras.optimizers.Adam(0.001))
  return model

### debug function after train -> go to csv

In [8]:
# from sklearn.model_selection import KFold
# import gc

# # 1. 读取原始数据 (加个 _raw 后缀以示区分)
# X_train_raw = pd.read_csv(TRAIN_FEATURE_PATH).iloc[:, 3:]
# y_train = pd.read_csv(TRAIN_LABEL_PATH)

# # 2. === 应用 SHB 特征工程 ===
# print(f"Original features: {X_train_raw.shape[1]}")
# print("Applying SHB features to training data...")
# X_train = add_shb_features(X_train_raw)

# # 3. === 关键：更新输入特征数量 ===
# # 覆盖掉之前硬编码的 (Numchannels * 2 + 4)
# global num_inputFeatures # 声明为全局变量以确保 designed_DNN_model 能读到正确的值
# num_inputFeatures = X_train.shape[1]
# print(f"Features updated. New input shape: {num_inputFeatures}")

# y_train.fillna(0, inplace=True)

# # K-Fold configuration
# n_splits = 5
# kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

# # Track the best model across all folds
# best_global_val_loss = float('inf')
# best_global_model_path = ""

# # K-Fold Training Loop
# fold_no = 1
# for train_index, val_index in kf.split(X_train):
#     # if fold_no != 5:
#     #     fold_no += 1
#     #     continue
#     print(f"Start Training Fold {fold_no}...")
    
#     # Initialize SwanLab for this fold
#     swanlab.init(
#         project="OFC_Challenge_SE_Block_WithFE",
#         experiment_name=f"SILUSEFold-{fold_no}",
#         config={
#             "fold": fold_no,
#             "learning_rate": "CosineAnnealing(Start=0.001)",
#             "architecture": "DNN+Residual+SE-Block",
#             "feature_engineering": "SHB_Features",
#             "epochs": 300,
#             "batch_size": 64, # Default
#         }
#     )

#     # 1. 内存清理
#     tf.keras.backend.clear_session()
#     gc.collect()

#     # 2. 数据切分
#     X_tr, X_val = X_train.iloc[train_index], X_train.iloc[val_index]
#     y_tr, y_val = y_train.iloc[train_index], y_train.iloc[val_index]

#     # 3. 初始化模型 (DNN)
#     base_model = designed_DNN_model(Numchannels, X_tr)
    
#     # === Cosine Annealing Learning Rate Schedule ===
#     epochs = 300
#     batch_size = 64
#     if len(X_tr) > 0:
#         steps_per_epoch = len(X_tr) // batch_size
#     else:
#         steps_per_epoch = 1 # Fallback safety
        
#     decay_steps = steps_per_epoch * epochs
    
#     lr_schedule = tf.keras.optimizers.schedules.CosineDecay(
#         initial_learning_rate=0.001,
#         decay_steps=decay_steps,
#         alpha=0.001  # Minimum LR = 0.001 * 0.001 = 1e-6
#     )
    
#     # 【修正 1】必须在这里编译！
#     base_model.compile(
#         loss=custom_loss_L2, 
#         optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule)
#     )

#     # 4. 回调函数
#     fold_model_path = f"{model_prepath}/ML_example_model_fold{fold_no}.h5"
    
#     checkpoint = tf.keras.callbacks.ModelCheckpoint(
#         filepath=fold_model_path,
#         monitor='val_loss',
#         save_best_only=True, # 只保存在这一折里表现最好的
#         mode='min',
#         save_weights_only=False,
#         verbose=0 # 设为 0 减少刷屏
#     )

#     early_stopping = tf.keras.callbacks.EarlyStopping(
#         monitor='val_loss',
#         patience=50,
#         restore_best_weights=True,
#         verbose=1
#     )
#         # 强烈建议加上这个，解决 Fold 2 震荡问题
# #     lr_reducer = tf.keras.callbacks.ReduceLROnPlateau(
# #             monitor='val_loss',
# #             factor=0.5,
# #             patience=20,
# #             min_lr=1e-6,
# #             verbose=1
# #   )  

#     # 5. 训练 DNN
#     print(f"   [DNN] Training with SE-Block + FE (CosineAnnealing)...")
#     history_dnn = base_model.fit(
#         X_tr, y_tr,
#         validation_data=(X_val, y_val),
#         verbose=2, 
#         epochs=epochs,
#         batch_size=batch_size,
#         callbacks=[checkpoint, early_stopping, SwanLabLogger()] # Add SwanLabLogger
#     )
#     print(f"   [DNN] Fold {fold_no} finished. Best model saved to {fold_model_path}")
    
#     fold_no += 1

# print("K-Fold Cross Validation Completed. All models are saved.")

In [9]:

# def plot_loss(indx,history,ingnoreIndex):
#     plt.figure(indx)
#     plt.plot(history.history['loss'][ingnoreIndex:], label='loss')
#     plt.plot(history.history['val_loss'][ingnoreIndex:], label='val_loss')
#     plt.xlabel('Epoch')
#     plt.ylabel('Error [gain]')
#     plt.legend()
#     plt.grid(True)
# plot_loss(1, history_dnn, 15)

In [ ]:
# === 生成伪标签并合并训练集 ===
print(">>> 开始生成伪标签 (Phase 1)...")

# 0. 加载测试数据 (与预测函数一致)
if not os.path.exists(TEST_FEATURE_PATH):
    print(f"Error: Test feature path not found: {TEST_FEATURE_PATH}")
else:
    X_test_full = pd.read_csv(TEST_FEATURE_PATH)
    X_test_raw = X_test_full.iloc[:, 5:]  # 保持切片一致
    print("Applying SHB features to test data...")
    X_test = add_shb_features(X_test_raw)
    print(f"Test features shape: {X_test.shape}")

# 0.5. 确保训练数据已加载 (如果未定义)
try:
    y_train
except NameError:
    print("Loading training labels...")
    y_train = pd.read_csv(TRAIN_LABEL_PATH)
    y_train.fillna(0, inplace=True)
    print(f"Training labels shape: {y_train.shape}")

try:
    X_train
except NameError:
    print("Loading and processing training features...")
    X_train_raw = pd.read_csv(TRAIN_FEATURE_PATH).iloc[:, 3:]
    print(f"Original features: {X_train_raw.shape[1]}")
    print("Applying SHB features to training data...")
    X_train = add_shb_features(X_train_raw)
    # 更新输入特征数量
    global num_inputFeatures
    num_inputFeatures = X_train.shape[1]
    print(f"Features updated. New input shape: {num_inputFeatures}")

# 1. 使用 5-Fold 模型预测测试集
n_folds = 4
pseudo_preds_list = []

for fold in range(1, n_folds + 1):
    fold_model_path = f"../model/sev_se_block_fe/ML_example_model_fold{fold}.h5"
    if os.path.exists(fold_model_path):
        model = tf.keras.models.load_model(fold_model_path, compile=False)
        pred = model.predict(X_test, batch_size=256, verbose=0)
        pseudo_preds_list.append(pred)
        print(f"Fold {fold} prediction done.")
    else:
        print(f"Warning: Model {fold_model_path} not found.")

if not pseudo_preds_list:
    print("Error: No models found for pseudo labeling.")
else:
    # 平均预测
    pseudo_preds = np.mean(pseudo_preds_list, axis=0)
    print(f"Ensemble prediction shape: {pseudo_preds.shape}")

    # 2. 整理数据格式
    pseudo_labels_df = pd.DataFrame(pseudo_preds, columns=y_train.columns)

    # 3. 过滤低置信度样本 (可选，这里简单保留所有)
    X_test_pseudo = X_test.copy()
    y_test_pseudo = pseudo_labels_df

    print(f"生成的伪标签样本数: {len(X_test_pseudo)}")

    print(">>> 合并训练集与伪标签数据 (Phase 2)...")

    # 1. 拼接
    X_combined = pd.concat([X_train, X_test_pseudo], axis=0).reset_index(drop=True)
    y_combined = pd.concat([y_train, y_test_pseudo], axis=0).reset_index(drop=True)

    print(f"原始训练集大小: {len(X_train)}")
    print(f"合并后训练集大小: {len(X_combined)}")

    # 2. 重新打乱数据
    from sklearn.utils import shuffle
    X_combined, y_combined = shuffle(X_combined, y_combined, random_state=42)

    print(">>> 伪标签生成和合并完成！")
    print(f"最终训练集大小: {len(X_combined)}")

>>> 开始生成伪标签 (Phase 1)...
Applying SHB features to test data...
  -> Performing Noise Clipping (< -55dBm to -60dBm)...
Test features shape: (21056, 216)
Loading training labels...


In [ ]:
# === 使用合并数据进行第二次训练 (Pseudo-Labeling Training) ===
from sklearn.model_selection import KFold
import gc
import tensorflow as tf
from tensorflow import keras

# SnapshotCallback 定义
class SnapshotCallback(keras.callbacks.Callback):
    def __init__(self, model_dir, n_cycles=5, total_epochs=300):
        self.model_dir = model_dir
        self.cycle_len = total_epochs // n_cycles
    def on_epoch_end(self, epoch, logs=None):
        if (epoch + 1) % self.cycle_len == 0:
            filename = f"{self.model_dir}/snapshot_model_epoch_{epoch+1}.h5"
            self.model.save(filename)
            print(f"Snapshot saved: {filename}")

# 使用合并后的数据进行训练
print(f"Using combined dataset: {X_combined.shape[0]} samples, {X_combined.shape[1]} features")

# K-Fold configuration
n_splits = 9
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

# Track the best model across all folds
best_global_val_loss = float('inf')
best_global_model_path = ""

# K-Fold Training Loop with Pseudo-Labels
fold_no = 1

for train_index, val_index in kf.split(X_combined):
    print(f"Start Training Fold {fold_no} with Pseudo-Labels...")

    # 1. 内存清理
    tf.keras.backend.clear_session()
    gc.collect()

    # 2. 数据切分 (使用合并数据)
    X_tr, X_val = X_combined.iloc[train_index], X_combined.iloc[val_index]
    y_tr, y_val = y_combined.iloc[train_index], y_combined.iloc[val_index]

    # 3. 初始化模型 (DNN)
    base_model = designed_DNN_model(Numchannels, X_tr)

    # === Cosine Annealing Learning Rate Schedule ===
    epochs = 300
    batch_size = 128
    if len(X_tr) > 0:
        steps_per_epoch = len(X_tr) // batch_size
    else:
        steps_per_epoch = 1

    decay_steps = steps_per_epoch * epochs

    lr_schedule = tf.keras.optimizers.schedules.CosineDecay(
        initial_learning_rate=0.001,
        decay_steps=decay_steps,
        alpha=0.001
)

    base_model.compile(
        loss=custom_loss_L2,
        optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule)
)

    # 4. 回调函数
    fold_model_path = f"{model_prepath}/ML_example_model_pseudo_fold{fold_no}.h5"

    checkpoint = tf.keras.callbacks.ModelCheckpoint(
        filepath=fold_model_path,
        monitor='val_loss',
        save_best_only=True,
        mode='min',
        save_weights_only=False,
        verbose=0
)

    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=20,
        restore_best_weights=True,
        verbose=1
)

    # === 新增：SnapshotCallback ===
    snapshot_cb = SnapshotCallback(model_prepath, n_cycles=5, total_epochs=300)

    # 5. 训练 DNN (使用伪标签数据)
    print(f"   [DNN] Training with SE-Block + FE + Pseudo-Labels (CosineAnnealing)...")
    history_dnn = base_model.fit(
        X_tr, y_tr,
        validation_data=(X_val, y_val),
        verbose=2,
        epochs=epochs,
        batch_size=batch_size,
        callbacks=[checkpoint, early_stopping, snapshot_cb]
)
    print(f"   [DNN] Fold {fold_no} finished. Best model saved to {fold_model_path}")

    fold_no += 1

print("Pseudo-Labeling K-Fold Cross Validation Completed. All models are saved.")

Using combined dataset: 160170 samples, 216 features
Start Training Fold 1 with Pseudo-Labels...
   [DNN] Training with SE-Block + FE + Pseudo-Labels (CosineAnnealing)...
Epoch 1/300
1113/1113 - 6s - loss: 0.2491 - val_loss: 0.2272 - 6s/epoch - 5ms/step
Epoch 2/300
1113/1113 - 5s - loss: 0.2217 - val_loss: 0.2094 - 5s/epoch - 4ms/step
Epoch 3/300
1113/1113 - 5s - loss: 0.2057 - val_loss: 0.1929 - 5s/epoch - 5ms/step
Epoch 4/300
1113/1113 - 5s - loss: 0.1810 - val_loss: 0.1702 - 5s/epoch - 5ms/step
Epoch 5/300
1113/1113 - 5s - loss: 0.1639 - val_loss: 0.1538 - 5s/epoch - 5ms/step
Epoch 6/300
1113/1113 - 5s - loss: 0.1539 - val_loss: 0.1599 - 5s/epoch - 4ms/step
Epoch 7/300
1113/1113 - 5s - loss: 0.1422 - val_loss: 0.1248 - 5s/epoch - 4ms/step
Epoch 8/300
1113/1113 - 5s - loss: 0.1329 - val_loss: 0.1207 - 5s/epoch - 4ms/step
Epoch 9/300
1113/1113 - 5s - loss: 0.1252 - val_loss: 0.1192 - 5s/epoch - 5ms/step
Epoch 10/300
1113/1113 - 4s - loss: 0.1250 - val_loss: 0.1158 - 4s/epoch - 4ms/ste

KeyboardInterrupt: 

In [ ]:
# === WRAPPED PREDICTION FUNCTION ===
def predict_with_tta(model, X_test, n_iter=10, noise_std=0.01):
    preds = []
    preds.append(model.predict(X_test))
    for i in range(n_iter):
        X_test_aug = X_test.copy()
        noise = np.random.normal(0, noise_std, X_test_aug.iloc[:, :95].shape)
        X_test_aug.iloc[:, :95] += noise
        preds.append(model.predict(X_test_aug))
    return np.mean(preds, axis=0)

from sklearn.neighbors import NearestNeighbors
import numpy as np
import os
import tensorflow as tf
from tensorflow import keras

def apply_wss_aware_knn(X_train, y_train, X_test, y_pred_test, alpha=0.3):
    wss_cols = [c for c in X_train.columns if 'DUT_WSS_activated_channel_index' in c]
    wss_cols.sort()
    def get_pattern_id(df):
        return df[wss_cols].astype(str).agg(''.join, axis=1)
    train_patterns = get_pattern_id(X_train)
    test_patterns = get_pattern_id(X_test)
    y_final = y_pred_test.copy()
    unique_patterns = test_patterns.unique()
    print(f"Found {len(unique_patterns)} unique WSS patterns in Test Set.")
    corrections_count = 0
    for pat in unique_patterns:
        train_indices = train_patterns[train_patterns == pat].index
        if len(train_indices) < 5:
            continue
        spectra_cols = [c for c in X_train.columns if 'EDFA_input_spectra' in c]
        X_train_sub = X_train.loc[train_indices, spectra_cols].values
        y_train_sub = y_train.loc[train_indices].values
        test_indices = test_patterns[test_patterns == pat].index
        X_test_sub = X_test.loc[test_indices, spectra_cols].values
        knn = NearestNeighbors(n_neighbors=min(10, len(X_train_sub)), metric='euclidean')
        knn.fit(X_train_sub)
        distances, neighbor_idx = knn.kneighbors(X_test_sub)
        for i, idx_in_test in enumerate(test_indices):
            neighbor_means = np.mean(y_train_sub[neighbor_idx[i]], axis=0)
            if distances[i][0] < 2.0:
                y_final[idx_in_test] = (1 - alpha) * y_final[idx_in_test] + alpha * neighbor_means
                corrections_count += 1
    print(f"Corrected {corrections_count} samples using WSS-Aware KNN.")
    return y_final

def run_ensemble_prediction():
    print(">>> FUNCTION START: run_ensemble_prediction")
    if not os.path.exists(TEST_FEATURE_PATH):
        print(f"Error: Test feature path not found: {TEST_FEATURE_PATH}")
        return
    X_test_full = pd.read_csv(TEST_FEATURE_PATH)
    X_test_raw = X_test_full.iloc[:, 5:]
    print("Applying SHB features to test data...")
    X_test = add_shb_features(X_test_raw)
    print(f"Test features shape: {X_test.shape}")
    n_splits = 4
    y_pred_ensemble = None
    ensemble_model_path = "../model/sev_se_block_fe/"
    TRAIN_LABEL_PATH = "../Features/Train/train_labels.csv"
    y_train = pd.read_csv(TRAIN_LABEL_PATH)
    TRAIN_FEATURE_PATH = "../Features/Train/train_features.csv"
    X_train_raw = pd.read_csv(TRAIN_FEATURE_PATH).iloc[:, 3:]
    X_train = add_shb_features(X_train_raw)
    print(f"--- Starting loop for {n_splits} folds from {ensemble_model_path} ---")
    for i in range(1, n_splits + 1):
        print(f"   Processing Fold {i}/{n_splits} ...")
        preds = []
        # 主模型
        fold_model_path = f"{ensemble_model_path}/ML_example_model_fold{i}.h5"
        if os.path.exists(fold_model_path):
            try:
                model_fold = tf.keras.models.load_model(fold_model_path, compile=False)
                pred_main = predict_with_tta(model_fold, X_test, n_iter=5, noise_std=0.01)
                preds.append(pred_main)
            except Exception as e:
                print(f"     [!] DNN Exception fold {i}: {e}")
        else:
            print(f"     [!] DNN Model file missing: {fold_model_path}")
        # 快照1
        snap1_path = f"{ensemble_model_path}/snapshot_model_epoch_60.h5"
        if os.path.exists(snap1_path):
            try:
                snap1_model = tf.keras.models.load_model(snap1_path, compile=False)
                pred_snap1 = predict_with_tta(snap1_model, X_test, n_iter=5, noise_std=0.01)
                preds.append(pred_snap1)
            except Exception as e:
                print(f"     [!] Snapshot Exception epoch 60: {e}")
        else:
            print(f"     [!] Snapshot file missing: {snap1_path}")
        # 快照2
        snap2_path = f"{ensemble_model_path}/snapshot_model_epoch_120.h5"
        if os.path.exists(snap2_path):
            try:
                snap2_model = tf.keras.models.load_model(snap2_path, compile=False)
                pred_snap2 = predict_with_tta(snap2_model, X_test, n_iter=5, noise_std=0.01)
                preds.append(pred_snap2)
            except Exception as e:
                print(f"     [!] Snapshot Exception epoch 120: {e}")
        else:
            print(f"     [!] Snapshot file missing: {snap2_path}")
        当前折的平均
        if preds:
            fold_pred = np.mean(preds, axis=0)
            if y_pred_ensemble is None:
                y_pred_ensemble = fold_pred
            else:
                y_pred_ensemble += fold_pred
    if y_pred_ensemble is None:
        print("Error: No predictions were made.")
        return
    y_pred_dnn = y_pred_ensemble / n_splits
    y_submission = apply_wss_aware_knn(X_train, y_train, X_test, y_pred_dnn, alpha=0.3)
    y_pred = pd.DataFrame(y_submission, columns=y_train.columns)
    wss_cols = [col for col in X_test.columns if 'dut_wss_activated_channel_index' in col.lower()]
    label_cols = [col for col in y_train.columns if 'calculated_gain_spectra' in col.lower()]
    mask = X_test[wss_cols].values == 1
    y_pred = pd.DataFrame(np.where(mask, y_pred.values, np.nan), columns=label_cols)
    y_pred.fillna(0, inplace=True)
    kaggle_ID = X_test_full.columns[0]
    y_pred.insert(0, kaggle_ID, X_test_full[kaggle_ID].values)
    output_path = "../Features/Test/submission_se_block_fe_snapshot_main2.csv"
    y_pred.to_csv(output_path, index=False)
    print(f"SUCCESS: Submission saved to {output_path}")
    print(">>> FUNCTION END")

run_ensemble_prediction()


>>> FUNCTION START: run_ensemble_prediction
Applying SHB features to test data...
  -> Performing Noise Clipping (< -55dBm to -60dBm)...
Test features shape: (21056, 216)
  -> Performing Noise Clipping (< -55dBm to -60dBm)...
--- Starting loop for 4 folds from ../model/sev_se_block_fe/ ---
   Processing Fold 1/4 ...
   Processing Fold 2/4 ...
   Processing Fold 3/4 ...
   Processing Fold 4/4 ...
Found 1090 unique WSS patterns in Test Set.
Corrected 0 samples using WSS-Aware KNN.
SUCCESS: Submission saved to ../Features/Test/submission_se_block_fe_snapshot_main2.csv
>>> FUNCTION END
